In [1]:
#installing library for transformer models
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
import os

Uploading local copy of the coyo-1k dataset to the IDE

In [4]:
from google.colab import files

In [6]:
uploaded=files.upload()

Saving coyo-1k.csv to coyo-1k.csv


In [7]:
df=pd.read_csv('coyo-1k.csv')

In [8]:
df.head()

,Unnamed: 0,url,key,image_phash,original_caption,tag,attribute,short_caption,detailed_caption,llm_caption
0,0,https://www.equipmentsalesandsurplus.com/v/vsp...,9113,e5989a6799687992,"Powermatic 60HH, 8"" Jointer with Helical Cutte...",['Jointer' 'Wood shaper' 'Thickness planer'],['repair which is a workbench or other surface...,['a table sawing machine on a white background'],['a planer that is sitting on a table'\r\n 'a ...,['A table sawing machine on an all-white backg...
1,1,https://texasfurniturehut.com/images/thumbs/00...,7371,be179168cce0cc97,Picture of ELIJAH LEATHER POWER RECLINER,['recliner' 'Power seat' 'Club chair'],['recliner which has upholstered in fabric or ...,['a brown leather recliner chair on a white ba...,['a brown recliner sitting in front of a white...,['A Brown Leather Recliner Chair On A White Ba...
2,2,https://cdn.shopify.com/s/files/1/1909/0703/pr...,1662,c04b03b43f1f3a67,"8 x 10 Reprinted Old Photo of U.S. Mail, Tanan...",['Dog sled' 'Sled dog' 'Sled dog racing'],['dog sled which has snow or ice'\r\n 'dog sle...,['a black and white photo of a dog team pullin...,['a man riding a dog sled past a post office'\...,['A black and white photo of a dog team pullin...
3,3,https://kelliesdesigns.net/wp-content/uploads/...,3390,f9e3c64e0cd8b036,Police Officer In His Hands Snap Tab,['Police officer' 'Police commissioner' 'Handc...,['policeman which has badge' 'cop which has ba...,['a pair of key chains with a police officer o...,"[""two key keychains with police officer's unif...","[""A pair of handmade key chains with a police ..."
4,4,https://cdn.houseplansservices.com/product/9ea...,3666,84cfbfb0c8cab20e,House Design - Colonial Exterior - Front Eleva...,['farmhouse' 'manufactured home'\r\n 'North am...,"['house which has exterior walls with siding, ...",['this is a computer rendering of a house'],['a two story brick house with a garage and po...,['This is a computer rendered image of a house...


In [9]:
df=df[['url','short_caption']]
df.head()

,url,short_caption
0,https://www.equipmentsalesandsurplus.com/v/vsp...,['a table sawing machine on a white background']
1,https://texasfurniturehut.com/images/thumbs/00...,['a brown leather recliner chair on a white ba...
2,https://cdn.shopify.com/s/files/1/1909/0703/pr...,['a black and white photo of a dog team pullin...
3,https://kelliesdesigns.net/wp-content/uploads/...,['a pair of key chains with a police officer o...
4,https://cdn.houseplansservices.com/product/9ea...,['this is a computer rendering of a house']


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
from tqdm import tqdm
import torch
from pyvis.network import Network

In [11]:
#loading model, tokenizer and defining generation parameters for the model
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

We define a function to extract triplets from the text present in the dataset using the model's output. This allows representation of information in a structured format that can be stored and queries in a graph database like Neo4j. 

In [12]:
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

By converting text data into triplets, we can store each triplet as a set of nodes connected by a relationship. This enables the running of complex queries on the extracted data, allowing for efficient and insightful analysis of relationships within the data.

In [13]:
# Text to extract triplets from
triplet_list=[]
url_list=[]

for i in tqdm(range(len(df))):
  text = df['short_caption'][i]
  url = df['url'][i]

  # Tokenizer text
  model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

  # Generate
  generated_tokens = model.generate(
      model_inputs["input_ids"].to(model.device),
      attention_mask=model_inputs["attention_mask"].to(model.device),
      **gen_kwargs,
  )

  # Extract text
  decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

  # Extract triplets
  for _, sentence in enumerate(decoded_preds):
      triplet_list.append(extract_triplets(sentence))
      url_list.append(url)


100%|██████████| 1000/1000 [57:33<00:00,  3.45s/it]


In [14]:
triplet_list[150:]

[[{'head': "'a christmas market in the middle of a town'",
   'type': 'instance of',
   'tail': 'christmas market'}],
 [{'head': 'market', 'type': 'facet of', 'tail': 'christmas'}],
 [{'head': 'market', 'type': 'part of', 'tail': 'town'}],
 [{'head': 'chips', 'type': 'part of', 'tail': 'sandwich'}],
 [{'head': 'sandwich', 'type': 'has part', 'tail': 'chips'}],
 [{'head': 'chips', 'type': 'subclass of', 'tail': 'sandwich'}],
 [{'head': 'salariess', 'type': 'subclass of', 'tail': 'wage'}],
 [{'head': 'wage', 'type': 'different from', 'tail': 'salariess'},
  {'head': 'salariess', 'type': 'different from', 'tail': 'wage'}],
 [{'head': 'wage', 'type': 'different from', 'tail': 'salariess'}],
 [{'head': 'dock', 'type': 'subclass of', 'tail': 'body of water'}],
 [{'head': 'dock', 'type': 'part of', 'tail': 'body of water'}],
 [{'head': 'dock', 'type': 'subclass of', 'tail': 'water'}],
 [{'head': 'pink', 'type': 'subclass of', 'tail': 'handbag'}],
 [{'head': 'pink', 'type': 'subclass of', 'tai

In [15]:
len(triplet_list)

3000

In [16]:
df_rebel=pd.DataFrame({'url':url_list,'triplet':triplet_list})
df_rebel.to_csv('rebel_babelscape_rebel-large.csv',index=False)
df_rebel.head()

,url,triplet
0,https://www.equipmentsalesandsurplus.com/v/vsp...,"[{'head': 'table sawing', 'type': 'subclass of..."
1,https://www.equipmentsalesandsurplus.com/v/vsp...,"[{'head': 'table sawing machine', 'type': 'sub..."
2,https://www.equipmentsalesandsurplus.com/v/vsp...,"[{'head': 'table sawing machine', 'type': 'has..."
3,https://texasfurniturehut.com/images/thumbs/00...,"[{'head': 'recliner', 'type': 'subclass of', '..."
4,https://texasfurniturehut.com/images/thumbs/00...,"[{'head': 'leather', 'type': 'subclass of', 't..."


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('url').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('url').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('url')):
  _plot_series(series, series_name, i)
  fig.legend(title='url', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_4['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_5['url'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_5, x='index', y='url', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [18]:
files.download('rebel_babelscape_rebel-large.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
import ast
import json

Selecting a portion of the DataFrame for further processing:

To facilitate better visualization and debugging, we are focusing on a specific subset of the data, specifically rows 201 to 300. This targeted approach allows us to closely inspect the triplets and their corresponding relationships in Neo4j.

In [24]:
selected_df = df_rebel.iloc[200:301]

In [21]:
new_data = {
    'n': [],
    'r': [],
    'm': []
}

In [31]:
for index, row in selected_df.iterrows():
        try:
            triplet = row['triplet']  # Directly get the list from the column
            if isinstance(triplet, str):
                triplet = eval(triplet)  # Convert string representation of list to actual list
            triplet_dict = triplet[0]  # Get the dictionary from the list

            head = triplet_dict['head']
            relation = triplet_dict['type']
            tail = triplet_dict['tail']

            new_data['n'].append(f"(:head {{name: '{head}'}})")
            new_data['r'].append(f"[:Relation {{type: {relation}}}]")
            new_data['m'].append(f"(:tail {{name: {tail}}})")
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing triplet at index {index}: {row['triplet']}")
            print(f"Exception: {e}")
            continue

In [32]:
new_df = pd.DataFrame(new_data)

In [ ]:
new_df.to_csv('transformed_126_to_151.csv', index=False)
print("Transformation complete. New CSV saved as 'transformed_201_to_300.csv'.")

In [34]:
files.download('transformed_201_to_300.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>